In [1]:
import pandas as pd
import numpy as np
import joblib
import sklearn
# import ensemble
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,balanced_accuracy_score,recall_score
import random
import lightgbm as lgb

In [2]:
import xgboost as xgb

In [3]:
data = pd.read_parquet("Data/Training/Amex_Campus_Challenge_Train.parquet")

In [4]:
randomlist = random.sample(list(data.customer.unique()),int(0.2*len(data.customer.unique())))

In [5]:
testdata = data[data['customer'].isin(randomlist)]

In [6]:
#select rows whose column value is not in a list
# traindata = data[~data['customer'].isin(randomlist)]
traindata = data

datarec = traindata[traindata['ind_recommended']==1].drop(['ind_recommended'], axis=1)
datanotrec = traindata[traindata['ind_recommended']==0].drop(['ind_recommended'], axis=1)

In [7]:
def confusion_matrix_2col(df, col1, col2):
    return df.groupby([col1, col2]).size().unstack(fill_value=0)

confusion_matrix_2col(traindata, 'ind_recommended', 'activation')

activation,0,1
ind_recommended,,
0,10622736,60173
1,1537226,9843


In [16]:
tdata = traindata.drop(traindata[traindata['ind_recommended'] == 0].sample(n=8622736).index)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(tdata.drop(['customer','merchant','activation','ind_recommended'],axis=1), tdata['ind_recommended'], test_size=0.2, random_state=1)
rec = lgb.LGBMClassifier()

In [18]:
rec.fit(X_train, y_train)
y_pred = rec.predict(X_test)
# print("mse : ",mean_squared_error(y_test, y_pred))

In [19]:
print("Accuracy : ",accuracy_score(y_test, y_pred),"Balanced Accuracy :",balanced_accuracy_score(y_test, y_pred)," recall :",recall_score(y_test, y_pred))

Accuracy :  0.6693113442530241 Balanced Accuracy : 0.6417881236161584  recall : 0.4480822050602643


In [20]:
traindata.groupby('activation').count()

,ind_recommended,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,customer_spend_03,...,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant
activation,,,,,,,,,,,,,,,,,,,,,
0,12159962,1444472,10113490,6313654,6313654,6313654,6313654,6313654,10113490,11217682,...,10639543,10639543,11574937,11758096,12079392,12127103,12155864,12159962,12159962,12159962
1,70016,28147,68723,62140,62140,62140,62140,62140,68723,69595,...,69776,69776,66325,69972,69832,69997,70003,70016,70016,70016


In [21]:
tdata = traindata.drop(traindata[traindata['activation'] == 0].sample(n=12059962).index)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(tdata.drop(['customer','merchant','activation','ind_recommended'],axis=1), tdata['activation'], test_size=0.2, random_state=1)
act = lgb.LGBMClassifier()

In [25]:
act.fit(X_train, y_train)
y_pred = act.predict(X_test)

In [26]:
print("Accuracy : ",accuracy_score(y_test, y_pred),"Balanced Accuracy :",balanced_accuracy_score(y_test, y_pred)," recall :",recall_score(y_test, y_pred))

Accuracy :  0.8707210916362781 Balanced Accuracy : 0.8678023725818835  recall : 0.8512278697886921


In [27]:
a = testdata[['ind_recommended','activation','customer','merchant']]
a.to_csv('d_test.csv', index=False)

In [28]:
try:
    testdata.drop(['rec','nrec','predicted_score'],axis=1,inplace=True)
except:
    pass

In [29]:
testdata

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant
0,0,0,NaN,107.215862,26.686594,74.0,3682.75,138.0,111.0,14.0,...,49466.0,65923.0,0.000000,29.180000,58.434969,32.500000,86.0,15.856826,168972,152285
2,0,0,NaN,31.623103,48.837872,19.0,2295.38,47.0,42.0,11.0,...,3912.0,12868.0,0.836364,421.500000,33.780445,0.000000,180.0,1.753009,225178,7439
10,0,0,NaN,196.890000,NaN,NaN,NaN,NaN,NaN,1.0,...,3682.0,2805.0,0.960000,14.861765,100.000000,0.500000,9.0,7.568990,159066,439468
11,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1142.0,607.0,0.076923,25.750000,0.000000,2.333333,25.0,4.970615,129516,87423
14,0,0,NaN,NaN,53.030000,1.0,53.03,1.0,1.0,NaN,...,1622.0,671.0,0.250000,116.070000,16.177457,2.833333,155.0,9.093101,252007,255993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12229964,0,1,1.0,69.532529,21.116111,9.0,1140.27,54.0,49.0,34.0,...,4643.0,9504.0,0.522727,25.287045,69.105417,7.833333,569.0,0.771669,22764,467964
12229967,0,1,171.0,45.115625,28.790419,13.0,4808.00,167.0,142.0,8.0,...,6328.0,7224.0,0.023256,20.700000,70.542237,3.500000,235.0,1.681623,22820,464989
12229969,0,1,1.0,50.297297,43.352879,35.0,2861.29,66.0,44.0,21.0,...,23212.0,56796.0,0.034483,43.130000,86.040035,6.833333,434.0,3.533709,22834,19907
12229973,0,1,10.0,41.706875,13.162500,9.0,368.55,28.0,19.0,5.0,...,7565.0,1171.0,NaN,17.568750,77.327287,3.166667,510.0,0.109332,23040,564879


In [30]:
rec_list = rec.predict_proba(testdata.drop(['ind_recommended','activation','customer','merchant'],axis=1))[:,1]
act_list = act.predict_proba(testdata.drop(['ind_recommended','activation','customer','merchant'],axis=1))[:,1]

In [32]:
#add column to df 
testdata['rec'] = list(rec_list)
testdata['act'] = list(act_list)

/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_9246/1902826599.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata['rec'] = list(rec_list)
/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_9246/1902826599.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata['act'] = list(act_list)


In [52]:
#testdata['predicted_score'] = testdata['rec']*(abs(1-testdata['nrec']))
testdata['predicted_score'] = (testdata['rec'].round()-0.5)*testdata['act']

/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_9246/512998719.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdata['predicted_score'] = (testdata['rec'].round()-0.5)*testdata['act']


In [53]:
testdata

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant,rec,act,predicted_score
0,0,0,NaN,107.215862,26.686594,74.0,3682.75,138.0,111.0,14.0,...,29.180000,58.434969,32.500000,86.0,15.856826,168972,152285,0.403893,0.682564,-0.341282
2,0,0,NaN,31.623103,48.837872,19.0,2295.38,47.0,42.0,11.0,...,421.500000,33.780445,0.000000,180.0,1.753009,225178,7439,0.199880,0.014729,-0.007365
10,0,0,NaN,196.890000,NaN,NaN,NaN,NaN,NaN,1.0,...,14.861765,100.000000,0.500000,9.0,7.568990,159066,439468,0.238778,0.030771,-0.015386
11,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.750000,0.000000,2.333333,25.0,4.970615,129516,87423,0.366360,0.015403,-0.007701
14,0,0,NaN,NaN,53.030000,1.0,53.03,1.0,1.0,NaN,...,116.070000,16.177457,2.833333,155.0,9.093101,252007,255993,0.478211,0.009728,-0.004864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12229964,0,1,1.0,69.532529,21.116111,9.0,1140.27,54.0,49.0,34.0,...,25.287045,69.105417,7.833333,569.0,0.771669,22764,467964,0.829066,0.961075,0.480538
12229967,0,1,171.0,45.115625,28.790419,13.0,4808.00,167.0,142.0,8.0,...,20.700000,70.542237,3.500000,235.0,1.681623,22820,464989,0.508437,0.946191,0.473096
12229969,0,1,1.0,50.297297,43.352879,35.0,2861.29,66.0,44.0,21.0,...,43.130000,86.040035,6.833333,434.0,3.533709,22834,19907,0.641997,0.826570,0.413285
12229973,0,1,10.0,41.706875,13.162500,9.0,368.55,28.0,19.0,5.0,...,17.568750,77.327287,3.166667,510.0,0.109332,23040,564879,0.429581,0.911216,-0.455608


In [54]:
testdata[['customer','predicted_score','merchant']].to_csv('d_pred.csv',index=False)

In [56]:
name='sidmethodwithform1'

In [57]:
import os
os.makedirs('submissions/'+name+'/models', exist_ok=True)

In [58]:
joblib.dump(rec, 'submissions/'+name+'/models/rec.joblib')
joblib.dump(act, 'submissions/'+name+'/models/act.joblib')

['submissions/sidmethodwithform1/models/act.joblib']

In [126]:
testdata.predicted_score.mean()

0.0037679773595457318

In [1]:
#form1  (rec.round-0.5)*act